# Notebook to analyse biomechanics data
## 04/10/2021
### Required python version: 3.6 

The following is a notebook that will carry out the biomechanics analysis for a single .csv file (can also accept .xlsx as input). It needs to be ran in the folder the data is in. 
<br>Required packages: os, pandas, numpy, terminaltables, sys, re, xlrd, matplotlib. Most installations of Anaconda will include pandas and numpy.
<br>Terminal tables from: https://anaconda.org/conda-forge/terminaltables
<br>
<br>If any packages aren't installed use package manager (preferably anaconda) to install, e.g.
```
conda install -c conda-forge terminaltables
```
<br>Contact Emily Johnson at ejohn16@liv.ac.uk or em.j.johnson.93@gmail.com if you're having trouble getting this notebook to work. 

In [41]:
# Load packages

import pandas as pd
import numpy as np
import os
import sys 
import re
import xlrd
from terminaltables import AsciiTable
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
import scipy.signal

First, create an empty dataframe for all the summary data to be saved to. This is less relevant when only analysing a single sample but in the batch script version it may contain thousands of samples. 

The results of this single sample analysis could be appended manually to a spreadsheet containing hundreds/thousands of samples. 

In [2]:
all_sample_summary = pd.DataFrame(columns=
    ['File name', 'Date', 'Sample ID', 'Replicate number', 'Sex', 'Age', 'Genotype', 
    'Sample length', 'Minimum force', 'Maximum force', 'Maximum force cycle 1', 'Maximum force cycle 5', 'Stress-relaxation', 'Rate of change of stress',
    'Hysteresis sum value', 'Hysteresis %', 'Smoothed hysteresis sum value', 'Smoothed hysteresis %', 
    'Average diameter', 'Circumference', 'Circumference, true', 'Max modulus', 'Stress at max modulus', 'Strain at max modulus', 'Failure stress (MPa)',
    'Failure strain (%)', 'Failure force (N)', 'Failure extension (mm)', 'Failure time (s)'])

A requirement for this notebook to run is a file containing all the meta-data for all the various samples: 'tendon_data_formatted.csv'. This processed meta-data file was created using the script 'format_sample_data.py', which extracted all the data from individual sheets and collated them into a flat .csv file. The original raw meta-data was in a file called 'Tm1b+oim Tendon Diameter +sampleInfo_270721.2.xlsx'. Thus, the next step is to check the processed meta-data file is accessible in the folder you're running the analysis in. 

In [3]:
# Get current working directory 
dir = os.getcwd()

# Check sample information file exists 
print("Checking to see if formatted sample meta-data file 'tendon_data_formatted.csv' is present in analysis directory...")

if os.path.isfile("tendon_data_formatted.csv"):
    print("Sample meta-data file found! Proceeding with analysis...")
    metadata = pd.read_csv("./tendon_data_formatted.csv", header=0)
    metadata = metadata.applymap(str)

else:
    sys.exit("Meta-data not found! Please make sure 'tendon_data_formatted.csv' is present in the same directory as the data. Terminating analysis...")

Checking to see if formatted sample meta-data file 'tendon_data_formatted.csv' is present in analysis directory...
Sample meta-data file found! Proceeding with analysis...


Assign the file you want to analyse to the variable 'file'. This can be a .csv file or a .xlsx file but a .csv file should be used by preference because they're quicker to read in.

To read in an .xlsx file run: 
```
file = str("210409 MRC Sample A1Data.xlsx")
df = pd.read_excel("{}".format(file), index_col=None, header=0)  
```

To read in a .csv file run:
```
df = pd.read_csv("210409 MRC Sample A1Data.csv", header=0)
```

In [4]:
df = pd.read_csv("210409 MRC Sample A1Data.csv", header=0)

The individual sample meta-data is extracted by pairing the file name to data in the meta-data file. To do this the file name is first divided into seperate variables, then these variables are used to locate the row containing the data.

In [5]:
name = os.path.splitext(file)[0]
excel_annotation = re.split('(\d+)', name)
sampleID = excel_annotation[2][-1]
dateID = excel_annotation[1]
replicateID = excel_annotation[3]
sample_metadata = metadata.loc[(metadata.Date_ID == dateID) & (metadata.Sample_ID == sampleID) & (metadata.Replicate == replicateID)]

Next, individual data frames are created for each stage of the tendon rupture experiment: pre-conditioning, stress-relaxation, failure.

You might consider these individual data frames the equivalent of the spreadsheets in the original excel anaylsis. 

To extract each data frame the column 'setname' is searched for a string matching the stage of the rupture experiment and each row matching the string is assigned to the new data frame.

In [6]:
precon = df[df['SetName'].str.contains('5x pre-conditioning')]
stressrelax = df[df['SetName'].str.contains('Stress-relax')]
failure = df[df['SetName'].str.contains('Failure')]

# df.iloc[:, 0:1] could also be used to select column 'SetName'

Next, the analysis is carried out for the pre-conditioning, stress-relaxation and failure data following the SOP.

In [7]:
## Pre-conditioning analysis - normalise/correct data

# Minimum force
minf = precon['Force_N'].min()

# Sample length
sample_length = precon.iloc[0,3]

# Load correction
precon['Load_correction'] = precon['Force_N'] - minf 
# Alternative: 
#precon['Load_correction'] = precon.iloc[:, 5] - minf

# Displacement correction 
precon['Displacement_correction'] = precon['Displacement_mm'] - minf 
# Alternative: 
#precon['Displacement_correction'] = precon.iloc[:, 4] - minf

# Area under curve
for j in range(0,precon.shape[0]-1):
    precon.loc[precon.index[j],'Area_under_curve'] = (0.1*(precon.iloc[j+1,6] + precon.iloc[j,6])*(precon.iloc[j+1,7] - precon.iloc[j,7]))

# Load correction smooth 
precon['Load_correction_smoothed'] = precon['Load_correction'].rolling(window=5).mean()

# Area under curve smooth
for j in range(0,precon.shape[0]-1):
    precon.loc[precon.index[j],'Area_under_curve_smooth'] = (0.1*(precon.iloc[j+1,9] + precon.iloc[j,9])*(precon.iloc[j+1,7] - precon.iloc[j,7]))


/home/ejohn16/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/home/ejohn16/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/home/ejohn16/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy

In [8]:
## Pre-conditioning analysis - stress relaxation

# Max force
maxforce = precon['Force_N'].max()

# Max force cycle 1 and cycle 5 
maxforce_c1 = precon.loc[precon.Cycle.str.contains('1'), 'Force_N'].max()
maxforce_c5 = precon.loc[precon.Cycle.str.contains('5'), 'Force_N'].max()

# Stress-relaxation 
stress_relaxation = (((maxforce_c1-maxforce_c5)/maxforce_c1)*100)

In [9]:
## Pre-conditioning analysis  - hysteresis 

# Hysteresis
# Sum of beginning of cycle 1 to last positive value in cycle 1
hysteresis_positive = precon.loc[(precon.Cycle.str.contains('1')) & (precon['Area_under_curve'] > 0), 'Area_under_curve'].sum()
# Sum of first negative value in cycle 5 to last negative value in cycle 5
hysteresis_negative = precon.loc[(precon.Cycle.str.contains('5')) & (precon['Area_under_curve'] < 0), 'Area_under_curve'].sum()
# Add the two together to calculate sum value
hysteresis_sum = hysteresis_positive + hysteresis_negative
# Then calculate percentage
percentage = (hysteresis_sum/hysteresis_positive)*100


# Hysteresis smooth 
# Repeat the same process but for the smoothed area under the curve values
smooth_hysteresis_positive = precon.loc[(precon.Cycle.str.contains('1')) & (precon['Area_under_curve_smooth'] > 0), 'Area_under_curve_smooth'].sum()
smooth_hysteresis_negative = precon.loc[(precon.Cycle.str.contains('5')) & (precon['Area_under_curve_smooth'] < 0), 'Area_under_curve_smooth'].sum()
smooth_hysteresis_sum = hysteresis_positive + hysteresis_negative
smooth_percentage = (hysteresis_sum/hysteresis_positive)*100



The stress-relaxation analysis is simple and only consists of one calculation across the dataframe.

In [10]:
## Stress-relaxation analysis 

stress_rate = ((stressrelax.iloc[0,5] - stressrelax.iloc[6000,5])/60)


For the failure analysis the data first needs to be normalised/corrected. 
This step requires the circumference data from the metadata file.

In [12]:
## Failure analysis - normalise/correct data

# Load correction
#failure['Load_correction'] = failure['Force_N'] - minf 
failure['Load_correction'] = failure.iloc[:, 5] - failure.iloc[0, 5]

# Displacement correction 
#failure['Displacement_correction'] = failure['Displacement_mm'] - minf 
failure['Displacement_correction'] = failure.iloc[:, 4] - failure.iloc[0, 4]

# Strain % 
failure['Strain_%'] = (failure['Displacement_correction']/sample_length)*100

# Strain (mm)
failure['Strain_mm'] = failure['Displacement_correction']/sample_length

# Stress (Mpas)
circumference_true = float(sample_metadata.iloc[0,9])
# make use of float fuction to convert string from dataframe into a float value (number with a decimal place)
failure['Stress_Mpas'] = failure['Load_correction']/circumference_true



/home/ejohn16/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/ejohn16/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ejohn16/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

In [13]:
## Failure analysis - modulus columns

# Need starting point for iteration in the modulus calculation
# To calculate, find where stretch phase begins
# Create index for whole failure sheet and just the stretch phase
stretch_index = failure.loc[(failure.Cycle.str.contains('Stretch'))]
stretch_index = np.array(pd.Index.tolist(stretch_index.index))
failure_index = np.array(pd.Index.tolist(failure.index))

# 'Stress @ 2positions before stretch as a moving value'
# Subtract the first value in the whole failure sheet index from the point where the stretch cycle starts, then subtract an addition 2 
modulus_start = (stretch_index[0] - failure_index[0]) - 2


# Modulus (Mpa)
for j in range(modulus_start,failure.shape[0]-5):
    failure.loc[failure.index[j],'Modulus_mpa'] = ((failure.iloc[j+5,10] - failure.iloc[j-4,10])/(failure.iloc[j+5,9] - failure.iloc[j-4,9]))


# Modulus - smooth 
failure['Modulus_smooth'] = failure['Modulus_mpa'].rolling(window=5).mean()


/home/ejohn16/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/ejohn16/anaconda3/envs/py36/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/ejohn16/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

For the failure calculations the point at which failure occurs is calculated using the max value in the 'Stress_mpas' column. 
After this, the failure data is subsetted to remove everything after failure has taken place - as any values taken after would not be valid for the next steps of the analysis.
From this newly subsetted data the max modulus value is calculated using the max value in the in 'Modulus_smooth' column. The smoothed modulus data is used because the unsmoothed data is too variable and a maximum value calculation would just calculate local behaviour rather than a true value.

In [37]:
## Failure analysis - modulus calculations 

# Calculate the stress value at which failure occurs
# Then use this to calculate strain, force and extension at which failure occurs
failure_stress = failure['Stress_Mpas'].max()
failure_strain_percent = failure.iloc[failure['Stress_Mpas'].argmax(), 8]
failure_force = failure.iloc[failure['Stress_Mpas'].argmax(), 6]
failure_extension = failure.iloc[failure['Stress_Mpas'].argmax(), 7]
failure_time = failure.iloc[failure['Stress_Mpas'].argmax(), 2]

#Subset failure column to remove everything after failure point for max modulus calculation
subset_f = failure.iloc[0:failure['Stress_Mpas'].argmax(), ]

# Calculate max modulus and stress/strain at max modulus on the newly subsetted version of the failure df
# (The full failure df will be the one saved at the end)
max_modulus = subset_f['Modulus_smooth'].max()
stress_at_max_modulus = subset_f.iloc[subset_f['Modulus_smooth'].argmax(), 10]
strain_at_max_modulus = subset_f.iloc[subset_f['Modulus_smooth'].argmax(), 9]

Now that the processing and analysis has been carried out the outputs can be saved.

In [18]:
## Create directory for output files

if not os.path.exists("{}/{}".format(dir, name)):
    os.makedirs("{}/{}".format(dir, name))

In [ ]:
## Pre-conditioning output 

# Pre-conditioning summary tables

force_data = [
    ['General summary', ''],
    ['Sample length', sample_length],
    ['Minimum force', minf],
    ['Maximum force', maxforce],
    ['Maximum force cycle 1', maxforce_c1],
    ['Maximum force cycle 5', maxforce_c5],
    ['Stress-relaxtion', stress_relaxation]
]
force_table = AsciiTable(force_data)

hysteresis_data = [
    ['Hysteresis', 'Cycl 1-5'],
    ['Positive value', hysteresis_positive],
    ['Sum value', hysteresis_sum],
    ['% value', percentage]
]
hysteresis_table = AsciiTable(hysteresis_data)


# Processed precon table as csv
precon.to_csv("{}/{}/precon_{}.csv".format(dir, name, name), index=False)


# Summary data as .txt file
with open("{}/{}/precon_summary_{}.txt".format(dir, name, name), 'w') as f:
    print("Summary data for {} preconditioning...\n".format(name), file=f)
    print(force_table.table, file=f) 
    print(hysteresis_table.table, file=f) 
    f.close()


In [40]:
## Failure output

# Failure summary table
modulus_data = [
    ['Summary', ''],
    ['Max modulus', max_modulus],
    ['Stress at max modulus', stress_at_max_modulus],
    ['Strain at max modulus', strain_at_max_modulus],
    ['Failure stress (MPa)', failure_stress],
    ['Failure strain (%)', failure_strain_percent],
    ['Failure force (N)', failure_force],
    ['Failure extension (mm)', failure_extension],
    ['Failure time (s)', failure_time]
]
modulus_table = AsciiTable(modulus_data)

# Processed failure table as csv
failure.to_csv("{}/{}/failure_{}.csv".format(dir, name, name), index=False)

# Summary data as .txt file
with open("{}/{}/failure_summary_{}.txt".format(dir, name, name), 'w') as f:
    print("Summary data for {} failure...\n".format(name), file=f)
    print(modulus_table.table, file=f) 
    f.close()

In [38]:
## Append current sample data to the overall summary
all_sample_summary = all_sample_summary.append({
    'File name': name, 
    'Date': sample_metadata.iloc[0,0], 
    'Sample ID': sample_metadata.iloc[0,1], 
    'Replicate number': sample_metadata.iloc[0,6], 
    'Sex': sample_metadata.iloc[0,3], 
    'Age': sample_metadata.iloc[0,4], 
    'Genotype': sample_metadata.iloc[0,5], 
    'Sample length': sample_length, 
    'Minimum force': minf, 
    'Maximum force': maxforce, 
    'Maximum force cycle 1': maxforce_c1, 
    'Maximum force cycle 5': maxforce_c5,
    'Stress-relaxation': stress_relaxation, 
    'Rate of change of stress': stress_rate, 
    'Hysteresis sum value': hysteresis_sum, 
    'Hysteresis %': percentage, 
    'Smoothed hysteresis sum value': smooth_hysteresis_sum,
    'Smoothed hysteresis %': smooth_percentage, 
    'Average diameter': sample_metadata.iloc[0,7], 
    'Circumference': sample_metadata.iloc[0,8], 
    'Circumference, true': sample_metadata.iloc[0,9],
    'Max modulus': max_modulus, 
    'Stress at max modulus': stress_at_max_modulus,
    'Strain at max modulus': strain_at_max_modulus, 
    'Failure stress (MPa)': failure_stress,
    'Failure strain (%)': failure_strain_percent,
    'Failure force (N)': failure_force,
    'Failure extension (mm)': failure_extension,
    'Failure time (s)': failure_time}, ignore_index=True)

In [ ]:
## Write summary table as output 
all_sample_summary.to_csv("{}/results_summary.csv".format(dir), index=False)

The individual sample analysis is now complete. After this some plotting could be carried out. Here is some sample code, that can be modified as you need. Specify your x and y parameters then plot everything on the same graph. 

In [45]:
# Plot code 

x = failure['Strain_%']
y1 = failure['Stress_Mpas']
yhat = scipy.signal.savgol_filter(y1, 101, 3) # window size 201, polynomial order 3

plt.figure()
plt.plot(x, y1, 'k-', label='Raw data', color='black', alpha=0.2) # set transparency with 'alpha' parameter
plt.plot(x, yhat, 'k-', label='Savgol', color='red')
plt.xlabel('Strain (%)')
plt.ylabel('Modulus (MPa)')
plt.title('Stress vs Strain',fontsize=12)
plt.legend()
plt.savefig('test.png'.format(dir, name, name), bbox_inches='tight',dpi=300)
plt.close()